# Lab 5 Text Clasification

Roshan Ahmed
20BRS1072

# Imports

In [1]:
!pip install torchtext
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import torchtext
import torch
from torch import nn,optim
from time import time

Defaulting to user installation because normal site-packages is not writeable


In [2]:
data = pd.read_csv("dataframe.csv")
print(data)



      Unnamed: 0                                               text  encoded
0              0  The soil I’m walking over comes    from deeper...    False
1              1  the close air of the earth whence she derived ...     True
2              2  Lyric night of the lingering Indian Summer,\nS...    False
3              3  “Percussus sum sicut foenum, et aruit cor meum...    False
4              4  I should be happy with my lot: A wife and moth...    False
...          ...                                                ...      ...
4789        4789  office has never really is its oldest daughter...     True
4790        4790  soot from her car with suitcases and hugged he...     True
4791        4791  Seeing in crowded restaurants the one you love...    False
4792        4792  flower sweet as I can never forsake And Fortun...     True
4793        4793  Echo that loved hid within a wood Would to her...    False

[4794 rows x 3 columns]


Creating tokenizer and vocabulary

In [3]:
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn.utils import rnn
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)


tokenizer = get_tokenizer("basic_english")

def build_vocabulary(datasets):
    for dataset in datasets:
        #print(dataset)
        for text in dataset['text']:
            yield tokenizer(text)

vocab = build_vocab_from_iterator(build_vocabulary([data,data]), min_freq=3, specials=["<UNK>"])

vocab.set_default_index(vocab["<UNK>"])

for l in data['text']:
  tokens = tokenizer(l)
  indexes = vocab(tokens)


Creating Dataloader

In [12]:

class Text(Dataset):
  def __init__(self,df):
    self.df = df
    #print(df)

  def pad(self,seq):
    if len(seq)>140:
      return seq[:140]
    else:
      for i in range(140-len(seq)):
        seq.insert(0,0)
      return seq

  def __len__(self):
    return len(self.df['encoded'])
  def __getitem__(self,idx):
    
    return torch.tensor(self.pad(vocab(tokenizer(self.df['text'][idx])))) , int(self.df['encoded'][idx])

test_loader  = DataLoader(Text(test.reset_index()) , batch_size=4)
train_loader = DataLoader(Text(train.reset_index()) , batch_size=128)

for txt , tgt in test_loader:
  _
  #print(txt)
  #print(txt)
voc = len(vocab)
#print(tokens)

print(vocab(["<UNK>"]))

[0]


In [13]:
print(test)
print(train)

      Unnamed: 0                                               text  encoded
3215        3215  farmerhad slammed doors were sweet potato but ...     True
3882        3882  to suggest someone had smashed and somehow hea...     True
2253        2253  Eat thou and drink; to-morrow thou shalt die. ...    False
385          385  O what a strange parcel of creatures are we, S...    False
4767        4767  The animals they mistake it that once upon the...     True
...          ...                                                ...      ...
3772        3772  growing too long. He takes from you. Not the k...     True
2916        2916  face. To wither in theyr love. It makes me sad...     True
991          991  do not say I thought I d pried too far beneath...     True
1307        1307  fright. I keep me alone. It My bad and we live...     True
3971        3971  O world, I cannot hold thee close enough!    T...    False

[959 rows x 3 columns]
      Unnamed: 0                                    

Creating RNN

In [23]:

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.embedding_layer = nn.Embedding(len(vocab),64)
        self.rnn = nn.RNN(64,32, 3, batch_first=True)
        self.linear1 = nn.Linear(32,32)
        self.linear2 = nn.Linear(32,2)
        self.act = nn.ReLU()
        self.out = nn.LogSoftmax(dim = 1)

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings)
        return self.out(self.linear2(self.act(self.linear1(self.act(output[:,-1])))))

model = RNN()

print(model)

RNN(
  (embedding_layer): Embedding(18658, 64)
  (rnn): RNN(64, 32, num_layers=3, batch_first=True)
  (linear1): Linear(in_features=32, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=2, bias=True)
  (act): ReLU()
  (out): LogSoftmax(dim=1)
)


In [24]:
accuracy = []
optimizer = optim.SGD(model.parameters(), lr=0.017, momentum=0.5)
criterion = nn.NLLLoss()
time0 = time()

Training

In [25]:
epochs = 20
for e in range(epochs):
    running_loss = 0
    for text, tgt in train_loader:
        # Training pass
        optimizer.zero_grad()

        output = model(text)

        loss = criterion(output, tgt)

        #This is where the model learns by backpropagating
        loss.backward()
        #And optimizes its weights here
        optimizer.step()
        #print(loss.item())
        running_loss += loss.item()

    else:
        #print("\nTraining Time (in minutes) =",(time()-time0)/60)
        #print(output)
        print("\nEpoch {} - Training loss: {}".format(e+1, running_loss/len(test_loader)))
    correct_count, all_count = 0, 0
    for images,labels in test_loader:
      for i in range(len(labels)):
        img = images[i].view(1,-1)
        with torch.no_grad():
          logps = model(img)


        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        #print(pred_label)
        if(true_label == pred_label):
          correct_count += 1
        all_count += 1

  #print("Number Of Images Tested =", all_count)
    print("Model Accuracy =", (correct_count/all_count))



Epoch 1 - Training loss: 0.04370230163137118
Model Accuracy = 0.4859228362877998

Epoch 2 - Training loss: 0.043363339950641
Model Accuracy = 0.48905109489051096

Epoch 3 - Training loss: 0.04311799928545952
Model Accuracy = 0.5088633993743483

Epoch 4 - Training loss: 0.04290950298309326
Model Accuracy = 0.5849843587069864

Epoch 5 - Training loss: 0.04270238156119983
Model Accuracy = 0.6454640250260688

Epoch 6 - Training loss: 0.04247122233112653
Model Accuracy = 0.6715328467153284

Epoch 7 - Training loss: 0.04219724237918854
Model Accuracy = 0.6871741397288843

Epoch 8 - Training loss: 0.041855471581220625
Model Accuracy = 0.6903023983315955

Epoch 9 - Training loss: 0.04141383717457454
Model Accuracy = 0.6903023983315955

Epoch 10 - Training loss: 0.040864415218432745
Model Accuracy = 0.6871741397288843

Epoch 11 - Training loss: 0.04022093539436658
Model Accuracy = 0.6923879040667362

Epoch 12 - Training loss: 0.039495383948087694
Model Accuracy = 0.6986444212721585

Epoch 13 -

#Texts for testing:

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

Even my brother is not like to speak with me. They treat me like aids patent.

